# Import libraries

In [ ]:
import os
import json
import sys
!{sys.executable} -m pip install oscar-python
!pip install liboidcagent
from oscar_python.client import Client

# GET TOKEN FROM EGI

In [ ]:
with open("/var/run/secrets/egi.eu/access_token") as f:
    access_token = f.read()

# Create OSCAR client with EGI TOKEN¶

In [ ]:
options_oidc = {'cluster_id':'oscar-egi-cloud',
            'endpoint':'https://inference.cloud.ai4eosc.eu',
            'oidc_token': access_token,
            'ssl':'True'}

oscar_client = Client(options=options_oidc)

# Read credentials

json_path = os.path.join("./credentials", os.environ.get("JSON_NAME") + ".json")
with open(json_path) as f:
    credentials = json.load(f)

# Create OSCAR client with the credentials

options_oidc = {'cluster_id': credentials['ID'],
            'endpoint': credentials['ENDPOINT'],
            'user': credentials['USER'],
            'password': credentials['PASSWORD'],
            'ssl': 'True'}

oscar_client = Client(options = options_oidc)

# Read text base 64 input

In [ ]:
image_name = os.environ.get("B64_INPUT")
output_file_name = os.getenv("DATA_OUTPUT")
output_file = f"./data/{image_name.rsplit('.', 1)[0]}.txt"

with open(output_file, "r") as txt_file:
    encoded_image = txt_file.read()

# Create oscar service: cowsay

In [ ]:
try:
    oscar_client.create_service("plant-classification-theano.yaml")
    print("Service created")
except Exception as ex:
    print("Error creating service: ", ex)

# Use the service

In [ ]:
response = oscar_client.run_service("plants", input=encoded_image)

# Verify if response is complete
if response.status_code == 200:
    # Process grayify service response
    data = response.text
    #  print("data:", data)
    os.makedirs('./output/b64', exist_ok=True)

    output_file_path = f"./output/b64/{output_file_name}"
    with open(output_file_path, 'w') as f:
        f.write(data)

else:
    print(f"Error in the execution of 'plants' service: {response.status_code} {response.text}")